# New York Taxi Trip

In [1]:
# Built-in module:
from os import getcwd, listdir, path
import sqlite3

# Data Manipulation modules:
import polars as pl
import numpy as np

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.style import use

use('ggplot')

## Setting paths to sqlite database:

In [2]:
NB_PATH = getcwd()
BASE = path.abspath(path.join(NB_PATH, '..'))
DATA = path.join(BASE, 'Data')
NYC_FP = path.join(DATA, 'NYC-TAXI-TRIP.parquet')

Loading all data from sqlite to into polars DataFrame then parse it and write it into parquet file for fast reading:

```python
DATA_2019 = path.join(DATA, '2019')
DBs = listdir(DATA_2019)
DFs = list()
schema = {'vendorid': pl.Int8, 'passenger_count': pl.Int32, 
          'trip_distance': pl.Float32, 'ratecodeid': pl.Int8, 'store_and_fwd_flag': pl.String, 'pulocationid': pl.Int32, 'dolocationid': pl.Int32,
          'payment_type': pl.Int8, 'fare_amount': pl.Float32, 'extra': pl.Float32, 'mta_tax': pl.Float32, 'tip_amount': pl.Float32, 
          'tolls_amount': pl.Float32, 'improvement_surcharge': pl.Float32, 'total_amount': pl.Float32, 'congestion_surcharge': pl.Float32}
for DB in DBs:
    DB_PATH = path.join(DATA_2019, DB)
    print(DB_PATH)
    SQL_QUERRY = "SELECT * FROM tripdata" # This is a SQL query to select tripdata and read it with pandas.
    with sqlite3.connect(DB_PATH) as con:
        TMP_DF = (
            pl
            .read_database(SQL_QUERRY, con, schema_overrides=schema)
        )
    DFs.append(TMP_DF)
    del TMP_DF
DF = (
    pl
    .concat(DFs)
    .with_columns([
        pl.col('tpep_pickup_datetime').str.to_datetime(),
        pl.col('tpep_dropoff_datetime').str.to_datetime()
    ])
)

del DFs
DF.write_parquet(NYC_FP)
del DF
```

In [3]:
DF = pl.read_parquet(NYC_FP)

## Cleaning:

### Inspection

In [4]:
DF.head()

vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
i8,datetime[μs],datetime[μs],i32,f32,i8,str,i32,i32,i8,f32,f32,f32,f32,f32,f32,f32,f32
1,2019-05-01 00:14:50,2019-05-01 00:16:48,1,0.0,1,"""N""",145,145,2,3.0,0.5,0.5,0.0,0.0,0.3,4.3,0.0
1,2019-05-01 00:35:54,2019-05-01 00:37:27,1,1.5,1,"""N""",145,145,2,3.0,0.5,0.5,0.0,0.0,0.3,4.3,0.0
1,2019-05-01 00:37:45,2019-05-01 00:37:49,1,1.5,1,"""N""",145,145,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0
1,2019-05-01 00:44:57,2019-05-01 00:50:11,1,0.7,1,"""N""",161,161,2,5.0,3.0,0.5,0.0,0.0,0.3,8.8,2.5
1,2019-05-01 00:59:48,2019-05-01 01:10:22,1,2.0,1,"""N""",163,141,1,9.5,3.0,0.5,2.0,0.0,0.3,15.3,2.5


### NAs Inspection:

In [5]:
DF.null_count()

vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
246601,0,0,246601,0,246601,246601,0,0,246601,0,0,0,0,0,0,0,4855981


### Adding Metadata:


In [6]:
vendorid = {1: 'Creative Mobile Technologies, LLC', 2: 'VeriFone Inc'}
ratecodeid = {1: 'Standard rate', 2: 'JFK', 3: 'Newark', 4:'Nassau or Westchester', 5:'Negotiated fare', 6:'Group ride'}
payment_type = {1: 'Credit card', 2: 'Cash', 3: 'No charge', 4: 'Dispute', 5: 'Unknown', 6: 'Voided trip'}

In [7]:
DF = (
    DF
    .with_columns([
        pl.col('tpep_pickup_datetime').dt.date().alias('tpep_pickup_date'),
        pl.col('tpep_pickup_datetime').dt.hour().alias('tpep_pickup_hour'),
        pl.col('tpep_pickup_datetime').dt.weekday().alias('tpep_pickup_weekday'),
        pl.col('tpep_dropoff_datetime').dt.date().alias('tpep_dropoff_date'),
        pl.col('tpep_dropoff_datetime').dt.hour().alias('tpep_dropoff_hour'),
        pl.col('tpep_dropoff_datetime').dt.weekday().alias('tpep_dropoff_weekday'),
        (pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_minutes().alias('trip_duration'),
        pl.col('vendorid').replace(vendorid).alias('vendor'),
        pl.col('ratecodeid').replace(ratecodeid).alias('ratecode'),
        pl.col('payment_type').replace(payment_type).alias('paymenttype')
    ])
    .drop('vendorid', 'ratecodeid', 'payment_type')
)

### Load taxi zone loockup table:

In [8]:
taxi_zones_loockup_fp = path.join(DATA, 'taxi_zone_lookup.csv')

taxi_zones = pl.read_csv(taxi_zones_loockup_fp, dtypes=[pl.Int32, pl.String, pl.String, pl.String])
print(f'taxi_zone shape: {taxi_zones.shape}')
taxi_zones.head()

taxi_zone shape: (265, 4)


LocationID,Borough,Zone,service_zone
i32,str,str,str
1,"""EWR""","""Newark Airport…","""EWR"""
2,"""Queens""","""Jamaica Bay""","""Boro Zone"""
3,"""Bronx""","""Allerton/Pelha…","""Boro Zone"""
4,"""Manhattan""","""Alphabet City""","""Yellow Zone"""
5,"""Staten Island""","""Arden Heights""","""Boro Zone"""


### Merge two datasets

Check for the key in the taxi zone table by which the merge will be done

In [9]:
(
    taxi_zones
    .select([
        pl.col('LocationID').n_unique(),
        pl.col('Borough').n_unique(),
        pl.col('Zone').n_unique(),
        pl.col('service_zone').n_unique()
    ])
)

LocationID,Borough,Zone,service_zone
u32,u32,u32,u32
265,7,262,5


We can see that LocationID column has only 260 unique values in a total of 263 rows it must contain either duplicates or null values we will first check for null values

In [10]:
taxi_zones.null_count()

LocationID,Borough,Zone,service_zone
u32,u32,u32,u32
0,0,0,0


We can see in the previous output that taxi_zone does not contain any null value so we ll check for duplicate values

In [12]:
DF = (
    DF
    .join(taxi_zones, left_on='pulocationid', right_on='LocationID')
    .join(taxi_zones, left_on='dolocationid', right_on='LocationID', suffix='_do')
    .rename({'Borough': 'puborough', 'Zone': 'puzone', 'service_zone': 'pu_service_zone', 'paymenttype': 'payment_type',
            'Borough_do': 'doborough', 'Zone_do': 'dozone', 'service_zone_do': 'do_service_zone'})
    .drop('pulocationid', 'bolocationid')
)

In [13]:
DF.head()

tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,store_and_fwd_flag,dolocationid,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,tpep_pickup_date,tpep_pickup_hour,tpep_pickup_weekday,tpep_dropoff_date,tpep_dropoff_hour,tpep_dropoff_weekday,trip_duration,vendor,ratecode,payment_type,puborough,puzone,pu_service_zone,doborough,dozone,do_service_zone
datetime[μs],datetime[μs],i32,f32,str,i32,f32,f32,f32,f32,f32,f32,f32,f32,date,i8,i8,date,i8,i8,i64,str,str,str,str,str,str,str,str,str
2019-05-01 00:14:50,2019-05-01 00:16:48,1,0.0,"""N""",145,3.0,0.5,0.5,0.0,0.0,0.3,4.3,0.0,2019-05-01,0,3,2019-05-01,0,3,1,"""Creative Mobil…","""Standard rate""","""Cash""","""Queens""","""Long Island Ci…","""Boro Zone""","""Queens""","""Long Island Ci…","""Boro Zone"""
2019-05-01 00:35:54,2019-05-01 00:37:27,1,1.5,"""N""",145,3.0,0.5,0.5,0.0,0.0,0.3,4.3,0.0,2019-05-01,0,3,2019-05-01,0,3,1,"""Creative Mobil…","""Standard rate""","""Cash""","""Queens""","""Long Island Ci…","""Boro Zone""","""Queens""","""Long Island Ci…","""Boro Zone"""
2019-05-01 00:37:45,2019-05-01 00:37:49,1,1.5,"""N""",145,2.5,0.5,0.5,0.0,0.0,0.3,3.8,0.0,2019-05-01,0,3,2019-05-01,0,3,0,"""Creative Mobil…","""Standard rate""","""Cash""","""Queens""","""Long Island Ci…","""Boro Zone""","""Queens""","""Long Island Ci…","""Boro Zone"""
2019-05-01 00:44:57,2019-05-01 00:50:11,1,0.7,"""N""",161,5.0,3.0,0.5,0.0,0.0,0.3,8.8,2.5,2019-05-01,0,3,2019-05-01,0,3,5,"""Creative Mobil…","""Standard rate""","""Cash""","""Manhattan""","""Midtown Center…","""Yellow Zone""","""Manhattan""","""Midtown Center…","""Yellow Zone"""
2019-05-01 00:59:48,2019-05-01 01:10:22,1,2.0,"""N""",141,9.5,3.0,0.5,2.0,0.0,0.3,15.3,2.5,2019-05-01,0,3,2019-05-01,1,3,10,"""Creative Mobil…","""Standard rate""","""Credit card""","""Manhattan""","""Midtown North""","""Yellow Zone""","""Manhattan""","""Lenox Hill Wes…","""Yellow Zone"""


In [14]:
DF.shape

(84399019, 30)

## EDA: